In [1]:
import jax
import jax.numpy as np
import jraph
import flax.linen as nn

from functools import partial

In [2]:
%load_ext autoreload
%autoreload 2

In [197]:
import sys
sys.path.append("../")

from models.gnn import GraphConvNet
from models.graph_utils import nearest_neighbors
from models.graph_utils import add_graphs_tuples

In [4]:
n_nodes = 5000
x = np.load("/n/holyscratch01/iaifi_lab/ccuesta/data_for_sid/halos.npy")[:, :n_nodes,:]

In [5]:
sources, targets = jax.vmap(nearest_neighbors, in_axes=(0, None))(x[:4], 20)

## EGNN

In [7]:
import jax.numpy as jnp
from jax import jit

def rotation_matrix(angle_deg, axis):
    angle_rad = jnp.radians(angle_deg)
    axis = axis / jnp.linalg.norm(axis)
    
    a = jnp.cos(angle_rad / 2)
    b, c, d = -axis * jnp.sin(angle_rad / 2)
    
    return jnp.array([
        [a*a + b*b - c*c - d*d, 2*(b*c - a*d), 2*(b*d + a*c)],
        [2*(b*c + a*d), a*a + c*c - b*b - d*d, 2*(c*d - a*b)],
        [2*(b*d - a*c), 2*(c*d + a*b), a*a + d*d - b*b - c*c]
    ])

@jit
def rotate_representation(data, angle_deg, axis):
    rot_mat = rotation_matrix(angle_deg, axis)
    
    positions = data[:, :3]
    velocities = data[:, 3:6]
    scalars = data[:, 6:]
    
    rotated_positions = jnp.matmul(rot_mat, positions.T).T
    rotated_velocities = jnp.matmul(rot_mat, velocities.T).T
    
    return jnp.concatenate([rotated_positions, rotated_velocities, scalars], axis=1)

In [66]:
x = (x - x.mean((0,1))) / x.std((0, 1))

In [412]:
from typing import Callable
import flax.linen as nn
import jax.numpy as jnp
import jraph

from models.graph_utils import add_graphs_tuples
from models.mlp import MLP

class CoordNorm(nn.Module):
    """ Coordinate normalization, from 
    https://github.com/lucidrains/egnn-pytorch/blob/main/egnn_pytorch/egnn_pytorch.py#LL67C28-L67C28
    """
    eps: float = 1e-8
    scale_init: float = 1.

    def setup(self):
        self.scale = self.param('scale', nn.initializers.ones, (1,))

    def __call__(self, coors):
        norm = jnp.linalg.norm(coors, axis=-1, keepdims=True)
        normed_coors = coors / jax.lax.clamp(self.eps, norm, jnp.inf)
        return normed_coors * self.scale

def get_edge_mlp_updates(d_hidden, n_layers, activation) -> Callable:
    """Get an edge MLP update function

    Args:
        mlp_feature_sizes (int): number of features in the MLP

    Returns:
        Callable: update function
    """

    def update_fn(
        edges: jnp.ndarray,
        senders: jnp.ndarray,
        receivers: jnp.ndarray,
        globals: jnp.ndarray,
    ) -> jnp.ndarray:
        """update edge features

        Args:
            edges (jnp.ndarray): edge attributes
            senders (jnp.ndarray): senders node attributes
            receivers (jnp.ndarray): receivers node attributes
            globals (jnp.ndarray): global features

        Returns:
            jnp.ndarray: updated edge features
        """
                
        # Split senders and receivers into coordinates, velocities, and scalar attrs
        x_i, v_i, h_i = senders[:, :3], senders[:, 3:6], senders[:, 6:]
        x_j, v_j, h_j = receivers[:, :3], receivers[:, 3:6], receivers[:, 6:]
                
        # Messages from Eqs. (3) and (4)/(7) 
        phi_e = MLP([d_hidden] * (n_layers), activation=activation)
        phi_x = MLP([d_hidden] * (n_layers - 1) + [1], activation=activation)
        
        m_ij =  phi_e(jnp.concatenate([h_i, h_j, jnp.linalg.norm(x_i - x_j, axis=1, keepdims=True) ** 2, globals], axis=-1))
        return (x_i - x_j) * phi_x(m_ij), m_ij

    return update_fn

def get_node_mlp_updates(d_hidden, n_layers, activation, n_edge) -> Callable:
    """Get an node MLP update function

    Args:
        mlp_feature_sizes (int): number of features in the MLP

    Returns:
        Callable: update function
    """

    def update_fn(
        nodes: jnp.ndarray,
        senders: jnp.ndarray,
        receivers: jnp.ndarray,
        globals: jnp.ndarray,
    ) -> jnp.ndarray:
        """update edge features

        Args:
            edges (jnp.ndarray): edge attributes
            senders (jnp.ndarray): senders node attributes
            receivers (jnp.ndarray): receivers node attributes
            globals (jnp.ndarray): global features

        Returns:
            jnp.ndarray: updated edge features
        """
        
        sum_x_ij, m_i = receivers  # Get aggregated messages
        x_i, v_i, h_i = nodes[:, :3], nodes[:, 3:6], nodes[:, 6:]  # Split node attrs
                
        # From Eqs. (6) and (7)
        phi_v = MLP([d_hidden] * (n_layers - 1) + [1], activation=activation)
        phi_h = MLP([d_hidden] * (n_layers - 1) + [h_i.shape[-1]], activation=activation)
                    
        # Apply updates
        v_i_p = sum_x_ij / (n_edge - 1) + phi_v(h_i) * v_i  # TODO: generalize connections
        x_i_p = x_i + v_i_p
        h_i_p = phi_h(jnp.concatenate([h_i, m_i], -1)) + h_i  # Skip connection
                
        return jnp.concatenate([x_i_p, v_i_p, h_i_p], -1)

    return update_fn

class GraphConvNet(nn.Module):
    """A simple graph convolutional network"""

    message_passing_steps: int = 4
    skip_connections: bool = False
    norm_layer: bool = True
    d_hidden: int = 64
    n_layers : int = 3
    activation: str = "swish"

    @nn.compact
    def __call__(self, graphs: jraph.GraphsTuple) -> jraph.GraphsTuple:
        """Do message passing on graph

        Args:
            graphs (jraph.GraphsTuple): graph object

        Returns:
            jraph.GraphsTuple: updated graph object
        """
        in_features = graphs.nodes.shape[-1]
        processed_graphs = graphs
        processed_graphs = processed_graphs._replace(globals=processed_graphs.globals.reshape(processed_graphs.globals.shape[0], -1))
        
        activation = getattr(nn, self.activation)
                
        update_node_fn = get_node_mlp_updates(self.d_hidden, self.n_layers, activation, n_edge=processed_graphs.n_edge)
        update_edge_fn = get_edge_mlp_updates(self.d_hidden, self.n_layers, activation)

        # Apply message-passing rounds
        for _ in range(self.message_passing_steps):
            
            graph_net = jraph.GraphNetwork(update_node_fn=update_node_fn, update_edge_fn=update_edge_fn)
            
            if self.skip_connections:
                processed_graphs = add_graphs_tuples(graph_net(processed_graphs), processed_graphs)
            else:
                processed_graphs = graph_net(processed_graphs)
            
            if self.norm_layer:
                processed_graphs = self.norm(processed_graphs)

        return processed_graphs
    
    def norm(self, graph):
        x, v, h = graph.nodes[..., :3], graph.nodes[..., 3:6], graph.nodes[..., 6:]
        
        # Only apply LN if scalars have more than one feature
        x, v, h = CoordNorm()(x), CoordNorm()(v), h if h.shape[-1] == 1 else nn.LayerNorm()(h)
        graph = graph._replace(nodes = jnp.concatenate([x, v, h], -1))
        return graph
        

In [413]:
class GraphWrapper(nn.Module):
    @nn.compact
    def __call__(self, x):
        model = jax.vmap(GraphConvNet())
        return model(x)
    
model = GraphWrapper()
rng = jax.random.PRNGKey(42)

## Test equivariance

In [414]:
graph = jraph.GraphsTuple(
          n_node=np.array(4 * [[n_nodes]]), 
          n_edge=np.array(4 * [[20]]),
          nodes=x[:4, :, :], 
          edges=None,
          globals=np.ones((4, 7)),
          senders=sources,
          receivers=targets)

graph_out, _ = model.init_with_output(rng, graph)

x_out = graph_out.nodes

angle_deg = 45.
axis = jnp.array([0, 1 / np.sqrt(2), 1 / np.sqrt(2)])

x_out_rot = jax.vmap(rotate_representation, in_axes=(0,None,None))(x_out, angle_deg, axis)

graph = jraph.GraphsTuple(
          n_node=np.array(4 * [[n_nodes]]), 
          n_edge=np.array(4 * [[20]]),
          nodes=jax.vmap(rotate_representation, in_axes=(0,None,None))(x[:4, :, :], angle_deg, axis),
          edges=None,
          globals=np.ones((4, 7)),
          senders=sources,
          receivers=targets)

graph_out, params = model.init_with_output(rng, graph)
x_out = graph_out.nodes

(1,) (5000, 3)
(1,) (5000, 3)
(1,) (5000, 3)
(1,) (5000, 3)
(1,) (5000, 3)
(1,) (5000, 3)
(1,) (5000, 3)
(1,) (5000, 3)


In [415]:
# Equivariance ratio
eq_ratio = x_out / x_out_rot
print(eq_ratio.max(), eq_ratio.min(), eq_ratio)

82.13094 -173.83261 [[[1.0035228  0.9978963  0.99760026 ... 0.9933725  1.0129737  0.9992706 ]
  [0.9998935  1.1863993  1.0017153  ... 1.0673395  1.0010754  1.0006332 ]
  [0.9999765  1.0002943  0.9997309  ... 1.000484   0.9994827  0.999958  ]
  ...
  [0.999844   1.0006032  1.0002435  ... 1.0005755  1.0000297  1.0003574 ]
  [1.0000621  0.99974406 1.0000896  ... 0.9997686  1.0004615  1.000152  ]
  [0.9994891  0.9999096  0.99968    ... 0.99943495 1.0002725  1.0003402 ]]

 [[1.0003672  0.9997328  1.0029218  ... 0.9998033  1.0028821  0.9992737 ]
  [0.99996704 1.0007361  0.999939   ... 1.0006534  0.99988943 1.0008017 ]
  [0.9999453  1.0414947  0.99876714 ... 1.4108531  0.9981786  1.0000074 ]
  ...
  [1.0001202  1.000273   0.9992023  ... 0.9983159  0.99890506 1.0005789 ]
  [0.9998512  1.000236   1.000517   ... 0.99991465 0.9340335  1.0002856 ]
  [1.0002218  1.0000648  1.0000988  ... 0.99110144 1.0042105  0.9992367 ]]

 [[1.00358    1.0215788  0.9995969  ... 1.0004034  0.9991566  0.9998411 ]
  

In [371]:
sum(x.size for x in jax.tree_util.tree_leaves(params))

119316